Importação de bibliotecas para abrir arquivos e medir o tempo de execução, além do multithreading

In [1]:
import time
import random
import threading
import logging

In [2]:
# # Gerar números aleatórios e salvar em um arquivo
# def gerar_numeros_aleatorios(arquivo: str, quantidade:int) -> None:
#     """
#     Gera uma quantidade especificada de números aleatórios e salva em um arquivo.
#     """
#     numeros = [str(random.randint(1, 10000)) for _ in range(quantidade)]
#     with open(arquivo, 'w') as f:
#         f.write(','.join(numeros))  # Usar espaço como separador
#     print(f"Números aleatórios gerados e salvos em {arquivo}")

### Implementação Insertion Sort
* O Insertion Sort é um algoritmo de ordenação que constrói a lista ordenada um elemento de cada vez.
* Ele é mais eficiente para listas pequenas ou listas que já estão quase ordenadas.
* O algoritmo funciona dividindo a lista em duas partes: a parte ordenada e a parte não ordenada.
* Ele pega um elemento da parte não ordenada e o insere na posição correta na parte ordenada.
* Varia entre O(n^2) e O(n) dependendo da ordenação inicial da lista.

In [3]:
def insertion_sort(lista: list) -> tuple[list, float]:
    """
    Ordena uma lista usando o algoritmo Insertion Sort.

    Args:
        lista: A lista a ser ordenada.

    Returns:
        Tupla com a lista ordenada e o tempo de execução.
    """
    if len(lista) == 0:
        return lista, 0.0
    
    inicio = time.time()
    
    for i in range(1, len(lista)):
        chave = lista[i]
        j = i - 1
        while j >= 0 and lista[j] > chave:
            lista[j + 1] = lista[j]
            j -= 1
        lista[j + 1] = chave
    
    fim = time.time()
    tempo_ordenacao = fim - inicio
    logging.info(f"Tempo total de execução do Insertion Sort: {tempo_ordenacao:.6f} segundos.")
    
    return lista, tempo_ordenacao

In [4]:
def insertion_sort_multithreaded(lista: list, num_threads: int, logger) -> tuple[list, float]:
    """
    Ordena uma lista usando o algoritmo Insertion Sort com múltiplas threads.

    Args:
        lista: A lista a ser ordenada.
        num_threads: O número de threads a serem usadas.
        logger: Logger para registrar informações.

    Returns:
        Tupla com a lista ordenada e o tempo de execução.
    """
    if num_threads <= 0:
        logger.error("O número de threads deve ser maior que zero.")
        raise ValueError("O número de threads deve ser maior que zero.")
    
    logger.info(f"Iniciando Insertion Sort com {num_threads} threads.")
    inicio = time.time()
    
    tamanho = len(lista)
    threads = []
    partes = [lista[i * tamanho // num_threads:(i + 1) * tamanho // num_threads] for i in range(num_threads)]

    # Função auxiliar para ordenar sem retornar tempo
    def insertion_sort_simples(sublista):
        for i in range(1, len(sublista)):
            chave = sublista[i]
            j = i - 1
            while j >= 0 and sublista[j] > chave:
                sublista[j + 1] = sublista[j]
                j -= 1
            sublista[j + 1] = chave

    for i in range(num_threads):
        thread = threading.Thread(target=insertion_sort_simples, args=(partes[i],))
        threads.append(thread)
        thread.start()
        logger.info(f"Thread {i + 1} iniciada para ordenar a parte {i + 1} com {len(partes[i])} elementos.")
    
    logger.info("Threads iniciadas para ordenação.")
    
    for thread in threads:
        thread.join()
    logger.info("Todas as threads concluídas.")

    # Combinar as partes ordenadas
    resultado = []
    for parte in partes:
        resultado.extend(parte)
    
    fim = time.time()
    tempo_ordenacao = fim - inicio
    logger.info(f"Tempo total de execução: {tempo_ordenacao:.6f} segundos.")
    
    return resultado, tempo_ordenacao

### Bucketsort
* Não é um método de ordenação, e sim um método de distribuição.
* O Bucket Sort é um algoritmo de ordenação que distribui os elementos em vários "baldes" (buckets).
* Cada balde é ordenado individualmente, geralmente usando um algoritmo de ordenação simples, como o Insertion Sort.
* O Bucket Sort é eficiente para listas com valores uniformemente distribuídos.
* Por exemplo, se tivermos uma lista entre 0 e 25, podemos criar 5 baldes: [0-5], [6-10], [11-15], [16-20], [20-25].
    * Para dividir os elementos, podemos usar a fórmula: bucket = elemento / 5
    * Arredondamos para cima, pois o bucket 0 é o [0-5], o bucket 1 é o [6-10], e assim por diante.
* Após a distribuição, ordenamos cada balde individualmente.
* Após a ordenação, concatenamos os baldes para obter a lista ordenada.

In [5]:
# Método Bucket Sort para auxiliar na ordenação
def separar_buckets(lista: list,qnt_buckets:int) -> list:
    """
    Separa os elementos da lista em buckets.

    Args:
        lista: A lista a ser separada.
        qnt_buckets: A quantidade de buckets a serem criados.

    Returns:
        A quantidade de buckets e a lista de buckets.
    """
    if len(lista) == 0:
        return []
    max_valor = max(lista)
    min_valor = min(lista)
    intervalo = (max_valor - min_valor) / qnt_buckets
    buckets = [[] for _ in range(qnt_buckets)]
    for numero in lista:
        # Determinar o índice do bucket
        if numero == max_valor:  # Garantir que o maior valor vá para o último bucket
            indice_bucket = qnt_buckets - 1
        else:
            indice_bucket = int((numero - min_valor) / intervalo)
        buckets[indice_bucket].append(numero)

    return buckets
    
def bucket_sort(lista: list, num_buckets: int) -> tuple[list, float]:
    """
    Ordena uma lista usando o algoritmo Bucket Sort.

    Args:
        lista: A lista a ser separada.
        num_buckets: A quantidade de buckets a serem criados.

    Returns:
        Tupla com a lista ordenada e o tempo de execução.
    """
    if len(lista) == 0:
        return lista, 0.0

    inicio = time.time()
    
    # Separar os elementos em buckets
    buckets = separar_buckets(lista, num_buckets)

    # Ordenar cada bucket individualmente (sem retorno de tempo)
    for i in range(len(buckets)):
        buckets[i], _ = insertion_sort(buckets[i])

    # Concatenar os buckets ordenados
    resultado = []
    for bucket in buckets:
        resultado.extend(bucket)
    
    fim = time.time()
    tempo_ordenacao = fim - inicio
    logging.info(f"Tempo total de execução do Bucket Sort: {tempo_ordenacao:.6f} segundos.")

    return resultado, tempo_ordenacao
  

In [6]:
def bucket_sort_multithreaded(lista: list, num_threads: int, logger, num_buckets: int) -> tuple[list, float]:
    """
    Ordena uma lista usando o algoritmo Bucket Sort com múltiplas threads.

    Args:
        lista: A lista a ser ordenada.
        num_threads: O número de threads a serem usadas.
        logger: Logger para registrar informações.
        num_buckets: Número de buckets a serem criados.

    Returns:
        Tupla com a lista ordenada e o tempo de execução.
    """
    if len(lista) == 0:
        return lista, 0.0

    if num_threads <= 0:
        logger.error("O número de threads deve ser maior que zero.")
        raise ValueError("O número de threads deve ser maior que zero.")
        
    logger.info(f"Iniciando Bucket Sort com {num_threads} threads.")
    inicio = time.time()
    
    # Separar os elementos em buckets
    buckets = separar_buckets(lista, num_buckets)
    logger.info(f"Separados {len(buckets)} buckets para ordenação.")
    
    # Função auxiliar para ordenar sem retornar tempo
    def insertion_sort_simples(sublista):
        for i in range(1, len(sublista)):
            chave = sublista[i]
            j = i - 1
            while j >= 0 and sublista[j] > chave:
                sublista[j + 1] = sublista[j]
                j -= 1
            sublista[j + 1] = chave

    # Dividir os buckets entre as threads
    def ordenar_grupo_buckets(buckets_grupo):
        for bucket in buckets_grupo:
            insertion_sort_simples(bucket)
    
    # Dividir os buckets em grupos para as threads
    buckets_por_thread = [[] for _ in range(num_threads)]
    for i, bucket in enumerate(buckets):
        buckets_por_thread[i % num_threads].append(bucket)
    
    threads = []
    for i in range(num_threads):
        thread = threading.Thread(target=ordenar_grupo_buckets, args=(buckets_por_thread[i],))
        threads.append(thread)
        thread.start()
        logger.info(f"Thread {i + 1} iniciada para ordenar {len(buckets_por_thread[i])} buckets.")
    
    for thread in threads:
        thread.join()
    logger.info("Todas as threads concluídas.")

    # Concatenar os buckets ordenados
    resultado = []
    for bucket in buckets:
        resultado.extend(bucket)
    
    fim = time.time()
    tempo_ordenacao = fim - inicio
    logger.info(f"Tempo total de execução: {tempo_ordenacao:.6f} segundos.")

    return resultado, tempo_ordenacao

In [7]:
# Abrir o arquivo "Numeros.txt" e ler os números
def abrir_arquivo(arquivo: str) -> list:
    """
    Lê os números de um arquivo e retorna uma lista.

    Args:
        arquivo: O caminho do arquivo a ser lido.

    Returns:
        Uma lista com os números lidos do arquivo.
    """
    with open(arquivo, 'r') as f:
        # Os numeros estão separados por vírgula
        numeros = f.read().strip().split('\n')
    # Converter os números de string para inteiro
    numeros = [int(num) for num in numeros if num.isdigit()]
    return numeros

In [ ]:
# Função para criar o arquivo de saída com os números ordenados, o tempo de execução e o método (Insertion Sort com ou sem bucketsort) e técnica de threading
def criar_arquivo_saida(arquivo_output:str, worA:str, num_elementos:int, tempo: float, metodo: str, multiThreading:bool, num_buckets:int,execucao:int)-> None:
    """
    Cria um arquivo de saída com os números ordenados, o tempo de execução e o método utilizado.

    Args:
        numeros: A lista de números ordenados.
        tempo: O tempo de execução do algoritmo.
        metodo: O método utilizado para ordenar os números.
    """
    with open(arquivo_output, worA) as f:
        # Verificar se o arquivo já existe
        if worA == 'w':
            f.write('Quantidade de Elementos,Tempo de Execucao (s),Metodo,Multithreading,Numero Buckets,Execução\n',)
        # Escrever os dados no arquivo
        f.write(f'{num_elementos},{tempo},{metodo},{"Sim" if multiThreading else "Nao"},{num_buckets},{execucao}\n')

In [9]:
def executar_ordenacao(algoritmo: str, usar_threads: bool = False, num_threads: int = 4, 
                       arquivo_entrada: str = '../Arquivos/input/numerosSimples.txt', 
                       arquivo_saida: str = '../Arquivos/output/outputSimples.csv',
                       modo_escrita: str = 'a',
                       num_buckets: int = 0,
                       execucao: int = 1) -> list:
    """
    Função generalizada para executar algoritmos de ordenação com ou sem multithreading.
    """
    # Configurar logger
    logger = logging.getLogger(f"{algoritmo}_{usar_threads}")
    logger.setLevel(logging.INFO)
    if not logger.handlers:
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    
    # Abrir o arquivo e ler os números
    numeros = abrir_arquivo(arquivo_entrada)
    print(f"Números lidos do arquivo: {len(numeros)} números")
    
    # Escolher o algoritmo e se deve usar multithreading
    if algoritmo.lower() == 'insertion':
        if usar_threads:
            logger.info(f"Usando {num_threads} threads para Insertion Sort.")
            numeros_ordenados, tempo_execucao = insertion_sort_multithreaded(numeros.copy(), num_threads, logger)
            metodo = 'Insertion Sort'
        else:
            logger.info("Usando Insertion Sort sem multithreading.")
            numeros_ordenados, tempo_execucao = insertion_sort(numeros.copy())
            metodo = 'Insertion Sort'
    elif algoritmo.lower() == 'bucket':
        if usar_threads:
            logger.info(f"Usando {num_threads} threads para Bucket Sort.")
            numeros_ordenados, tempo_execucao = bucket_sort_multithreaded(numeros.copy(), num_threads, logger, num_buckets)
            metodo = 'Bucket Sort'
        else:
            logger.info("Usando Bucket Sort sem multithreading.")
            numeros_ordenados, tempo_execucao = bucket_sort(numeros.copy(), num_buckets)
            metodo = 'Bucket Sort'
    else:
        logger.error(f"Algoritmo {algoritmo} não reconhecido.")
        raise ValueError(f"Algoritmo {algoritmo} não reconhecido. Use 'insertion' ou 'bucket'.")
    
    print(f"Tempo de execução: {tempo_execucao:.6f} segundos")
    
    # Criar o arquivo de saída com os resultados
    criar_arquivo_saida(arquivo_saida, modo_escrita, len(numeros), tempo_execucao, metodo, usar_threads, num_buckets,execucao)
    
    return numeros_ordenados

In [ ]:
# Criar um lock global
ordenacao_lock = threading.Lock()
def executar_todos_metodos(execucao:int):
    """
    Executa todos os métodos de ordenação e salva os resultados.
    Usa um lock para garantir que apenas um método seja executado por vez.
    
    Args:
        arquivo_entrada: O caminho do arquivo de entrada.
        arquivo_saida: O caminho do arquivo de saída.
    """
    dirs_entrada = {
        'Aleatorios': '../Arquivos/input/Aleatorios',
        'Ordenados': '../Arquivos/input/Ordenados',
        'Decrescentes': '../Arquivos/input/Decrescentes',
        'Parcialmente Ordenados': '../Arquivos/input/ParcialmenteOrdenados'
    }
    
    arquivos_entradas = {}
    #
    num_elementos = [100,200,500,1000,2000,5000,7500,10000,15000,30000,50000,75000,100000,200000,500000,750000,1000000,1250000,1500000,2000000]
    for tipo, caminho in dirs_entrada.items():
        for i in num_elementos:
            inicio = tipo.lower()[0] if ' ' not in tipo else tipo.lower().split(" ")[0][0] + tipo.lower().split(" ")[1][0]        
            arquivo = f'{caminho}/{inicio+str(i)}.txt'
            arquivos_entradas[f'{tipo} {i}'] = arquivo
    
    


    # Adquirir o lock global para garantir execução exclusiva
    with ordenacao_lock:
        print("Iniciando execução de todos os métodos de ordenação...")
        
        for tipo, arquivo_entrada in arquivos_entradas.items():
            print(type(tipo))
            print(f"\n=== Executando métodos para {tipo[0]} ===")
            # Criar o arquivo de saída específico para cada tipo
            arquivo_saida = f'../Arquivos/output/output_{tipo.split(" ")[0]}.csv'
            
            # Insertion Sort sem threads
            print("\n=== Executando Insertion Sort sem threads ===")
            executar_ordenacao('insertion', False, arquivo_entrada=arquivo_entrada, 
                               arquivo_saida=arquivo_saida,execucao=execucao)
            
            # Insertion Sort com threads
            print("\n=== Executando Insertion Sort com threads ===")
            executar_ordenacao('insertion', True, arquivo_entrada=arquivo_entrada, 
                               arquivo_saida=arquivo_saida,execucao=execucao)
            
            # Bucket Sort sem threads, 10 buckets
            print("\n=== Executando Bucket Sort sem threads ===")
            executar_ordenacao('bucket', False, arquivo_entrada=arquivo_entrada, 
                              arquivo_saida=arquivo_saida,num_buckets=10,execucao=execucao)
        
            # Bucket Sort sem threads, 100 buckets
            print("\n=== Executando Bucket Sort sem threads ===")
            executar_ordenacao('bucket', False, arquivo_entrada=arquivo_entrada, 
                              arquivo_saida=arquivo_saida,num_buckets=100,execucao=execucao)
            
            # Bucket Sort sem threads, 1000 buckets
            print("\n=== Executando Bucket Sort sem threads ===")
            executar_ordenacao('bucket', False, arquivo_entrada=arquivo_entrada, 
                              arquivo_saida=arquivo_saida,num_buckets=1000,execucao=execucao)
            
            # Bucket Sort com threads, 10 buckets
            print("\n=== Executando Bucket Sort com threads ===")
            executar_ordenacao('bucket', True, arquivo_entrada=arquivo_entrada, 
                              arquivo_saida=arquivo_saida, num_buckets=10,execucao=execucao)
            # Bucket Sort com threads, 100 buckets
            print("\n=== Executando Bucket Sort com threads ===")
            executar_ordenacao('bucket', True, arquivo_entrada=arquivo_entrada, 
                              arquivo_saida=arquivo_saida, num_buckets=100,execucao=execucao)
            # Bucket Sort com threads, 1000 buckets
            print("\n=== Executando Bucket Sort com threads ===")
            executar_ordenacao('bucket', True, arquivo_entrada=arquivo_entrada, 
                              arquivo_saida=arquivo_saida, num_buckets=1000,execucao=execucao)
        
        
        print("\nTodos os métodos de ordenação foram executados com sucesso.")

In [11]:
for i in range(1, 6):
    print(f"Executando teste {i}...")
    executar_todos_metodos(i)

2025-07-07 15:28:18,754 - insertion_False - INFO - Usando Insertion Sort sem multithreading.
2025-07-07 15:28:18,758 - insertion_True - INFO - Usando 4 threads para Insertion Sort.
INFO:insertion_True:Usando 4 threads para Insertion Sort.
2025-07-07 15:28:18,763 - insertion_True - INFO - Iniciando Insertion Sort com 4 threads.
INFO:insertion_True:Iniciando Insertion Sort com 4 threads.
2025-07-07 15:28:18,769 - insertion_True - INFO - Thread 1 iniciada para ordenar a parte 1 com 25 elementos.
INFO:insertion_True:Thread 1 iniciada para ordenar a parte 1 com 25 elementos.
2025-07-07 15:28:18,775 - insertion_True - INFO - Thread 2 iniciada para ordenar a parte 2 com 25 elementos.
INFO:insertion_True:Thread 2 iniciada para ordenar a parte 2 com 25 elementos.
2025-07-07 15:28:18,780 - insertion_True - INFO - Thread 3 iniciada para ordenar a parte 3 com 25 elementos.
INFO:insertion_True:Thread 3 iniciada para ordenar a parte 3 com 25 elementos.
2025-07-07 15:28:18,785 - insertion_True - INFO

Executando teste 1...
Iniciando execução de todos os métodos de ordenação...
<class 'str'>

=== Executando métodos para A ===

=== Executando Insertion Sort sem threads ===
Números lidos do arquivo: 100 números
Tempo de execução: 0.001003 segundos

=== Executando Insertion Sort com threads ===
Números lidos do arquivo: 100 números
Tempo de execução: 0.032537 segundos

=== Executando Bucket Sort sem threads ===
Números lidos do arquivo: 100 números
Tempo de execução: 0.001001 segundos

=== Executando Bucket Sort sem threads ===
Números lidos do arquivo: 100 números
Tempo de execução: 0.000998 segundos

=== Executando Bucket Sort sem threads ===
Números lidos do arquivo: 100 números
Tempo de execução: 0.001000 segundos

=== Executando Bucket Sort com threads ===
Números lidos do arquivo: 100 números
Tempo de execução: 0.034015 segundos

=== Executando Bucket Sort com threads ===
Números lidos do arquivo: 100 números
Tempo de execução: 0.029998 segundos

=== Executando Bucket Sort com thr

2025-07-07 15:28:18,931 - bucket_True - INFO - Thread 1 iniciada para ordenar 250 buckets.
INFO:bucket_True:Thread 1 iniciada para ordenar 250 buckets.
2025-07-07 15:28:18,935 - bucket_True - INFO - Thread 2 iniciada para ordenar 250 buckets.
INFO:bucket_True:Thread 2 iniciada para ordenar 250 buckets.
2025-07-07 15:28:18,940 - bucket_True - INFO - Thread 3 iniciada para ordenar 250 buckets.
INFO:bucket_True:Thread 3 iniciada para ordenar 250 buckets.
2025-07-07 15:28:18,945 - bucket_True - INFO - Thread 4 iniciada para ordenar 250 buckets.
INFO:bucket_True:Thread 4 iniciada para ordenar 250 buckets.
2025-07-07 15:28:18,949 - bucket_True - INFO - Todas as threads concluídas.
INFO:bucket_True:Todas as threads concluídas.
2025-07-07 15:28:18,953 - bucket_True - INFO - Tempo total de execução: 0.029001 segundos.
INFO:bucket_True:Tempo total de execução: 0.029001 segundos.
2025-07-07 15:28:18,959 - insertion_False - INFO - Usando Insertion Sort sem multithreading.
INFO:insertion_False:Usan

Tempo de execução: 0.029001 segundos
<class 'str'>

=== Executando métodos para A ===

=== Executando Insertion Sort sem threads ===
Números lidos do arquivo: 200 números
Tempo de execução: 0.002000 segundos

=== Executando Insertion Sort com threads ===
Números lidos do arquivo: 200 números
Tempo de execução: 0.029532 segundos

=== Executando Bucket Sort sem threads ===
Números lidos do arquivo: 200 números
Tempo de execução: 0.000999 segundos

=== Executando Bucket Sort sem threads ===
Números lidos do arquivo: 200 números
Tempo de execução: 0.001003 segundos

=== Executando Bucket Sort sem threads ===
Números lidos do arquivo: 200 números
Tempo de execução: 0.000998 segundos

=== Executando Bucket Sort com threads ===
Números lidos do arquivo: 200 números
Tempo de execução: 0.030012 segundos

=== Executando Bucket Sort com threads ===
Números lidos do arquivo: 200 números


INFO:bucket_True:Thread 2 iniciada para ordenar 25 buckets.
2025-07-07 15:28:19,161 - bucket_True - INFO - Thread 3 iniciada para ordenar 25 buckets.
INFO:bucket_True:Thread 3 iniciada para ordenar 25 buckets.
2025-07-07 15:28:19,168 - bucket_True - INFO - Thread 4 iniciada para ordenar 25 buckets.
INFO:bucket_True:Thread 4 iniciada para ordenar 25 buckets.
2025-07-07 15:28:19,172 - bucket_True - INFO - Todas as threads concluídas.
INFO:bucket_True:Todas as threads concluídas.
2025-07-07 15:28:19,175 - bucket_True - INFO - Tempo total de execução: 0.093021 segundos.
INFO:bucket_True:Tempo total de execução: 0.093021 segundos.
2025-07-07 15:28:19,194 - bucket_True - INFO - Usando 4 threads para Bucket Sort.
INFO:bucket_True:Usando 4 threads para Bucket Sort.
2025-07-07 15:28:19,203 - bucket_True - INFO - Iniciando Bucket Sort com 4 threads.
INFO:bucket_True:Iniciando Bucket Sort com 4 threads.
2025-07-07 15:28:19,208 - bucket_True - INFO - Separados 1000 buckets para ordenação.
INFO:buc

...


=== Executando Bucket Sort sem threads ===
Números lidos do arquivo: 15000 números
Tempo de execução: 0.008528 segundos

=== Executando Bucket Sort sem threads ===
Números lidos do arquivo: 15000 números
Tempo de execução: 0.013049 segundos

=== Executando Bucket Sort com threads ===
Números lidos do arquivo: 15000 números
Tempo de execução: 0.035460 segundos

=== Executando Bucket Sort com threads ===
Números lidos do arquivo: 15000 números
Tempo de execução: 0.026177 segundos

=== Executando Bucket Sort com threads ===
Números lidos do arquivo: 15000 números
Tempo de execução: 0.027185 segundos
<class 'str'>

=== Executando métodos para P ===

=== Executando Insertion Sort sem threads ===
Números lidos do arquivo: 30000 números
Tempo de execução: 0.069666 segundos


2025-07-07 15:43:58,371 - insertion_True - INFO - Usando 4 threads para Insertion Sort.
INFO:insertion_True:Usando 4 threads para Insertion Sort.
2025-07-07 15:43:58,375 - insertion_True - INFO - Iniciando Insertion Sort com 4 threads.
INFO:insertion_True:Iniciando Insertion Sort com 4 threads.
2025-07-07 15:43:58,389 - insertion_True - INFO - Thread 1 iniciada para ordenar a parte 1 com 7500 elementos.
INFO:insertion_True:Thread 1 iniciada para ordenar a parte 1 com 7500 elementos.
2025-07-07 15:43:58,404 - insertion_True - INFO - Thread 2 iniciada para ordenar a parte 2 com 7500 elementos.
INFO:insertion_True:Thread 2 iniciada para ordenar a parte 2 com 7500 elementos.
2025-07-07 15:43:58,411 - insertion_True - INFO - Thread 3 iniciada para ordenar a parte 3 com 7500 elementos.
INFO:insertion_True:Thread 3 iniciada para ordenar a parte 3 com 7500 elementos.
2025-07-07 15:43:58,422 - insertion_True - INFO - Thread 4 iniciada para ordenar a parte 4 com 7500 elementos.
INFO:insertion_Tr


=== Executando Insertion Sort com threads ===
Números lidos do arquivo: 30000 números
Tempo de execução: 0.061215 segundos

=== Executando Bucket Sort sem threads ===
Números lidos do arquivo: 30000 números
Tempo de execução: 0.027571 segundos

=== Executando Bucket Sort sem threads ===
Números lidos do arquivo: 30000 números
Tempo de execução: 0.015034 segundos

=== Executando Bucket Sort sem threads ===
Números lidos do arquivo: 30000 números
Tempo de execução: 0.016162 segundos


2025-07-07 15:43:58,565 - bucket_True - INFO - Usando 4 threads para Bucket Sort.
INFO:bucket_True:Usando 4 threads para Bucket Sort.
2025-07-07 15:43:58,568 - bucket_True - INFO - Iniciando Bucket Sort com 4 threads.
INFO:bucket_True:Iniciando Bucket Sort com 4 threads.
2025-07-07 15:43:58,581 - bucket_True - INFO - Separados 10 buckets para ordenação.
INFO:bucket_True:Separados 10 buckets para ordenação.
2025-07-07 15:43:58,592 - bucket_True - INFO - Thread 1 iniciada para ordenar 3 buckets.
INFO:bucket_True:Thread 1 iniciada para ordenar 3 buckets.
2025-07-07 15:43:58,600 - bucket_True - INFO - Thread 2 iniciada para ordenar 3 buckets.
INFO:bucket_True:Thread 2 iniciada para ordenar 3 buckets.
2025-07-07 15:43:58,605 - bucket_True - INFO - Thread 3 iniciada para ordenar 2 buckets.
INFO:bucket_True:Thread 3 iniciada para ordenar 2 buckets.
2025-07-07 15:43:58,612 - bucket_True - INFO - Thread 4 iniciada para ordenar 2 buckets.
INFO:bucket_True:Thread 4 iniciada para ordenar 2 buckets


=== Executando Bucket Sort com threads ===
Números lidos do arquivo: 30000 números
Tempo de execução: 0.045868 segundos

=== Executando Bucket Sort com threads ===
Números lidos do arquivo: 30000 números
Tempo de execução: 0.032141 segundos

=== Executando Bucket Sort com threads ===
Números lidos do arquivo: 30000 números
Tempo de execução: 0.034581 segundos

Todos os métodos de ordenação foram executados com sucesso.
